determining the optic disk position

In [ ]:
import os
from PIL import Image
import numpy as np
import cv2 as cv


# Portions of this code were generated with the assistance of ChatGPT (OpenAI, 2025) and subsequently modified by the author.
# OpenAI. (2025). ChatGPT (May 2025 version) [Large language model]. https://chat.openai.com

In [ ]:

#Apply a trained model on large image

testDirImage = "D:/Thesis/datasets/BrG_training_data/glaucoma/"
testDirMask = "D:/Thesis/datasets/BrG_training_data/glaucoma_mask_disc/"

large_og_images = []
large_test_images = []
other_images = []
large_masks = []

# for path in os.listdir(testDirImage2):
#     if path.endswith('.png'):
#         img = Image.open(testDirImage2 + path)
#         red, green, blue = img.split()
#         img = np.asarray(green)
#         print(img.shape)
#         large_test_images += [img]

for path in os.listdir(testDirImage):
    if path.endswith('.png'):
        img = cv.imread(cv.samples.findFile(testDirImage + path))
        im = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        large_og_images += [im]
        red, green, blue = cv.split(im)
        other_images += [green]
        img = cv.cvtColor(img,cv.COLOR_BGR2HSV)
        H, S, V = cv.split(img)
        img = np.asarray(V)

        #print(img.shape)
        large_test_images += [img]

for path in os.listdir(testDirMask):
    if path.endswith('.png'):
        im = cv.imread(cv.samples.findFile(testDirMask + path))
        large_masks += [im]

random = np.random.randint(0,len(large_test_images))
print(random)
large_test_image = large_test_images[random]
print(np.array(large_test_image).shape)

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(np.array(large_og_images[random]))
axes[1].imshow(np.array(other_images[random]), cmap='gray')
axes[2].imshow(np.array(large_test_image), cmap='gray')  # Assuming the first image is grayscale
plt.show()
#patches = patchify(large_test_image, (256, 256), step=256)  #Step=256 for 256 patches means no overlap

In [ ]:
import random
def crop_to_circle(img):
    """Crops the image into a perfect circle and removes the background completely."""
    h, w = img.shape[:2]
    
    # Create a circular mask
    mask = np.zeros((h, w), dtype=np.uint8)
    center = (w // 2, h // 2)
    radius = min(w, h) // 2
    cv.circle(mask, center, radius, 255, -1)

    # Apply mask to keep only circular region
    img_circular = cv.bitwise_and(img, img, mask=mask)

    # Convert to image with an alpha (transparency) channel
    final_img = cv.cvtColor(img_circular, cv.COLOR_BGR2BGRA)  # Maintain original colors
    final_img[:, :, 3] = mask  # Set transparency using the mask

    return img_circular

def apply_gaussian_blur(img):
    #return cv2.GaussianBlur(img, (3, 3), 1.2)
    #return cv.GaussianBlur(img, (11, 11), 7)  # Increased from (3,3), 1.2
    #return cv.GaussianBlur(img, (21, 21), 14)  # Increased from (3,3), 1.2
    return cv.GaussianBlur(img, (11, 11), 7)  # Increased from (3,3), 1.2
    #return cv.GaussianBlur(img, (5, 5), 2.4)  # Increased from (3,3), 1.2

def add_lens_flare(img):

    h, w = img.shape[:2]
    flare = np.zeros_like(img, dtype=np.uint8)

    # Randomly position the flare anywhere in the image
    center_x = random.randint(int(0.4 * w), int(0.6 * w))  
    center_y = random.randint(int(0.4 * h), int(0.6 * h))  
    #center_x, center_y = w // 2, h // 2

    # Define flare properties
    radius = random.randint(25, 40)
    intensity = random.uniform(0.2, 0.4)
    color = (255, 255, 255)

    # Draw the soft flare
    cv.circle(flare, (center_x, center_y), radius, color, -1, cv.LINE_AA)

    # Blend the flare effect smoothly
    return cv.addWeighted(img, 1, flare, intensity, 0)


def darken_and_redden_image(img, darkness_factor=0.6, red_boost=1, contrast_alpha=1.2):

    if img is None:
        print("Error: image not found.")
        return

    # Convert to float32 for safe manipulation
    img = img.astype(np.float32)

    # Step 1: Darken the image
    img *= darkness_factor

    # Step 2: Boost red channel (OpenCV uses BGR format!)
    img[:, :, 2] *= red_boost  # Red channel is index 2

    # Clip values to [0, 255] and convert back to uint8
    img = np.clip(img, 0, 255).astype(np.uint8)

    img = cv.convertScaleAbs(img, alpha=contrast_alpha, beta=0)  # alpha > 1 increases contrast

    return img

In [ ]:
testDirImage = "D:/Thesis/datasets/BrG_test_data/normal/"
testDirMask = "D:/Thesis/datasets/BrG_test_data/normal_mask_disc/"
for path in os.listdir(testDirImage):
    if path.endswith('.png'):
        img = cv.imread(cv.samples.findFile(testDirImage + path))
        mask = cv.imread(cv.samples.findFile(testDirMask + path))
        im = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        large_og_images += [im]
        red, green, blue = cv.split(im)
        other_images += [green]
        img = cv.cvtColor(img,cv.COLOR_BGR2HSV)
        H, S, V = cv.split(img)
        img = np.asarray(V)

        # Find the brightest value (max pixel value)
        brightest_value = np.max(img)
        print("Brightest pixel value:", brightest_value)

        # Find coordinates of all brightest pixels
        brightest_pixels = np.column_stack(np.where(img >= brightest_value-40))

        # OPTIONAL: Filter out outliers using DBSCAN clustering
        from sklearn.cluster import DBSCAN

        # Cluster the bright pixels
        clustering = DBSCAN(eps=10, min_samples=5).fit(brightest_pixels)

        # Get the labels (each cluster gets a label: 0, 1, 2, etc., and -1 means noise)
        labels = clustering.labels_

        # Pick the largest cluster
        unique_labels, counts = np.unique(labels[labels != -1], return_counts=True)
        if len(counts) > 0:
            biggest_cluster_label = unique_labels[np.argmax(counts)]
            dense_pixels = brightest_pixels[labels == biggest_cluster_label]
        else:
            dense_pixels = brightest_pixels  # fallback

        # Calculate the center (centroid) of the dense bright pixels
        center_y = int(np.mean(dense_pixels[:, 0]))
        center_x = int(np.mean(dense_pixels[:, 1]))

        # Calculate fixed square boundaries
        half_size = 256 // 2
        x_min = center_x - half_size
        x_max = center_x + half_size
        y_min = center_y - half_size
        y_max = center_y + half_size

        # Make sure the square doesn't go outside image boundaries
        height, width = img.shape
        if x_min < 0 :
            x_max = x_max + abs(x_min)
            x_min = 0
        if y_min < 0 :
            y_max = y_max + abs(y_min)
            y_min = 0
        if x_max > width:
            x_min = x_min - (x_max - width)
            x_max = width
        if y_max > height:
            y_min = y_min - (y_max - height)
            y_max = height

        # 🟰 CROP the image
        cropped_img = im[y_min:y_max, x_min:x_max]
        cropped_mask = mask[y_min:y_max, x_min:x_max]

        print("Cropped image shape:", cropped_img.shape)

        #cv.imwrite('D:/Thesis/datasets/Retina_blood_vessel_kaggle/Data/train/cropped_im/'+str(i)+"_im.png", cv.cvtColor(cropped_img, cv.COLOR_RGB2BGR))

        #cropped_img = apply_gaussian_blur(cropped_img)
        #cropped_img = add_lens_flare(cropped_img)
        #cropped_img = darken_and_redden_image(cropped_img)
        #cropped_img = crop_to_circle(cropped_img)
        #cropped_mask = crop_to_circle(cropped_mask)

        cv.imwrite('D:/Thesis/datasets/BrG_test_data/cropped_normal_2/'+path, cv.cvtColor(cropped_img, cv.COLOR_RGB2BGR))
        cv.imwrite('D:/Thesis/datasets/BrG_test_data/cropped_normal_disc_2/'+path, cropped_mask)

In [ ]:
testDirImage = "D:/Thesis/datasets/BrG_test_data/all_im/"
#testDirImage = "D:/Thesis/datasets/kaggle_drive/train/image/"

large_og_images = []

for path in os.listdir(testDirImage):
    if path.endswith('.png'):
        img = cv.imread(cv.samples.findFile(testDirImage + path))
        im = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        im = apply_gaussian_blur(im)
        im = add_lens_flare(im)
        #im = darken_and_redden_image(im)
        #im = crop_to_circle(im)

        cv.imwrite('D:/Thesis/datasets/BrG_test_data/all_bad_im/'+path, cv.cvtColor(im, cv.COLOR_RGB2BGR))

In [ ]:
import os
import cv2 as cv
import numpy as np
def crop_image(image):
    h, w,_ = image.shape

    # Desired crop size
    crop_width = 256
    crop_height = 256

    # Calculate top-left corner of the crop
    start_x = (w - crop_width) // 2
    start_y = (h - crop_height) // 2

    # Ensure the crop stays within image bounds
    start_x = max(0, start_x)
    start_y = max(0, start_y)
    end_x = start_x + crop_width
    end_y = start_y + crop_height

    # Crop the center
    return image[start_y:end_y, start_x:end_x]

testDirImage = "D:/Thesis/datasets/BrG_test_data/normal/"
#testDirImage = "D:/Thesis/datasets/kaggle_drive/train/image/"
good_size = []
for path in os.listdir(testDirImage):
    if path.endswith('.png'):
        img = cv.imread(cv.samples.findFile(testDirImage + path))
        im = cv.cvtColor(img,cv.COLOR_BGR2RGB)
        im = cv.resize(crop_image(im), (256, 256))
        # im = apply_gaussian_blur(im)
        # im = add_lens_flare(im)
        #im = darken_and_redden_image(im)
        #im = crop_to_circle(im)
        cv.imwrite('D:/Thesis/datasets/BrG_test_data/cropped_normal/'+path, cv.cvtColor(im, cv.COLOR_RGB2BGR))

In [ ]:
import os
import glob
import shutil

# Paths to your folders
source_folder = "D:/Thesis/datasets/BrG_training_data/normal_mask_disc"
reference_folder = "D:/Thesis/datasets/BrG_training_data/all_im"
destination_folder = "D:/Thesis/datasets/BrG_training_data/normal_mask_disc_test"

# Ensure destination exists
os.makedirs(destination_folder, exist_ok=True)

# Get filenames from the reference folder
reference_names = set(os.path.basename(f) for f in glob.glob(os.path.join(reference_folder, "*")))

# Loop through source files
for filepath in glob.glob(os.path.join(source_folder, "*")):
    filename = os.path.basename(filepath)
    if filename not in reference_names:
        print(f"Moving: {filename}")
        shutil.move(filepath, os.path.join(destination_folder, filename))